In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/study/16715robotsim/project/Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using OrdinaryDiffEq
using Test
using Plots

In [3]:
const g = 9.8

m = 1
μ = 0.5

w = 1.0 # width of the box
h = 0.8 # height of the box
M = [m 0 0; 0 m 0; 0 0 m*((h/2)^2+(w/2)^2)/3]

# set of contact modes
modes = [0 0 0 0; # both free
        1 0 0 0; # sticking
        1 0 -1 0; # left-slide
        1 0 1 0; # right-slide
        0 1 0 0;
        0 1 0 -1;
        0 1 0 1;
        1 1 0 0;
        1 1 -1 -1;
        1 1 1 1]

q0 = [0;0.5;0] # x y θ
dq0 = [0;0;0]
x0 = [q0;dq0]
initial_mode = [0 0 0 0]

n_contacts = 2


2

In [4]:
function R_2D(θ)
    R = [cos(θ) -sin(θ); sin(θ) cos(θ)]
    return R
end

R_2D (generic function with 1 method)

In [5]:
# constraints/contacts
function compute_a(q)
    y = q[2]
    θ = q[3]
    a1 = y - 0.5*h*cos(θ) - 0.5*w*sin(θ)
    a2 = y - 0.5*h*cos(θ) + 0.5*w*sin(θ)
    return [a1; a2]
end

function compute_a_t(q)
    p1 = R_2D(q[3])*[-w/2; -h/2] 
    p2 = R_2D(q[3])*[w/2; -h/2] 
    at1 = p1[1] + q[1]
    at2 = p2[1] + q[1]
    return [at1; at2]
end

# constraints jacobian
function compute_A(q)
    θ = q[3]
    A = [0 1 0.5*h*sin(θ)-0.5*w*cos(θ); 0 1 0.5*h*sin(θ)+0.5*w*cos(θ)]
    return A
end

function compute_dA(q, dq)
    dA = reshape(ForwardDiff.jacobian(compute_A, q)*dq, n_contacts, 3)
    return dA
end

function compute_A_tangent(q)
    A_t = ForwardDiff.jacobian(compute_a_t,q)
    return A_t
end

function compute_dA_tangent(q, dq)
    dA_t = reshape(ForwardDiff.jacobian(compute_A_tangent, q)*dq, n_contacts, 3)
    return dA_t
end

compute_dA_tangent (generic function with 1 method)

In [6]:
function contact_mode_constraints(x, contactMode)
    
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    A_all = zeros(0,3)
    A_all_f = zeros(0,3)
    dA_all = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            A_all_f = [A_all_f; A[k,:]'; A_t[k,:]']
            dA_all = [dA_all; dA[k,:]'; dA_t[k,:]']
            A_all = [A_all; A[k,:]'; A_t[k,:]']
        else
            A_all_f = [A_all_f; A[k,:]'-ss*μ*A_t[k,:]']
            dA_all = [dA_all; dA[k,:]']
            A_all = [A_all; A[k,:]']
        end
    end
    
    return A_all_f, A_all, dA_all
end

contact_mode_constraints (generic function with 1 method)

In [19]:
function contact_force_constraints(λ, contactMode)
    # c > 0
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    ss_active = ss_mode[cs_mode]
    
    i = 1
    ic = 1
    c = zeros(sum(ss_active.==0)*3 + sum(ss_active.!=0))
    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            c[ic:ic+2] = [-λ[i]; -μ*λ[i] - λ[i+1]; -μ*λ[i] + λ[i+1]]
            ic += 3
            i += 2
        else
            c[ic] = -λ[i]
            i += 1
            ic += 1
        end
    end
    return c
end

contact_force_constraints (generic function with 1 method)

In [21]:
function sliding_velocity_constraints(x, contactMode)
    # reture A_eq, A_geq, A_eq*dq = 0, A_geq*dq >= 0
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    AA_eq = zeros(0,3)
    AA_geq = zeros(0,3)
    dAA_eq = zeros(0,3)
    dAA_geq = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            AA_eq = [AA_eq; A[k,:]'; A_t[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]'; dA_t[k,:]']
        else
            AA_eq = [AA_eq; A[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]']
            AA_geq = [AA_geq; ss*A_t[k,:]']
            dAA_geq = [dAA_geq; ss*dA_t[k,:]']
        end
    end
    
    return AA_eq, AA_geq, dAA_eq, dAA_geq
end

sliding_velocity_constraints (generic function with 1 method)

In [22]:
function solveEOM(x, contactMode, controller)
    # contactMode: bool vector, indicates which constraints are active
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    Y = controller(x, contactMode)
    
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    #z = blockMat\b
    #println(blockMat)
    if rank(blockMat) < length(b)
        z =pinv(blockMat)*b
    else
        z = blockMat\b
    end
    
    ddq = z[1:3]
    if (sum(contactMode[1:n_contacts])>=1)
        λ = z[4:end]
    else
        λ = []
    end
    
    return ddq, λ
end

solveEOM (generic function with 1 method)

In [24]:
function computeResetMap(x, contactMode)
    q = x[1:3]
    dq = x[4:6]

    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    c = size(A, 1)
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 
    
    if rank(blockMat) < 3+c
        z = pinv(blockMat)*[M*dq; zeros(c)]
    else
        z = blockMat\[M*dq; zeros(c)]
    end
        
    dq_p = z[1:3]
    p_hat = z[4:end]
    return dq_p, p_hat
end

computeResetMap (generic function with 1 method)

In [26]:
function compute_FA(x, controller)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< 1e-6
    inactive_cs = abs.(a) .> 1e-6
    
    if sum(active_cs) == 0
        return zeros(Int, 4)
    end
    
    possibleModes = zeros(Bool, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end

    max_cons = 0
    
    for kk = 1:size(possibleModes, 1)      
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, dA_separate = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        ddq, λ = solveEOM(x, m, controller)
        
        c_λ = contact_force_constraints(λ, m)
        
        if all(c_λ.>=0)
        
            As_eq, As_geq, dAs_eq, dAs_geq = sliding_velocity_constraints(x, m)

            sep_vel_cond = ((A_separate*dq).>0) .| ((A_separate*ddq .+ dA_separate*dq).>=0)
            maintain_vel_cond = all(abs.(As_eq*dq).<1e-6) & all((As_geq*dq).>1e-6)

            if ~maintain_vel_cond
                if any((As_geq*dq).<1e-6)
                    maintain_vel_cond = all(abs.(dAs_eq*dq + As_eq*ddq).<1e-6) & all((dAs_geq*dq + As_geq*ddq).>0)
                end
            end

            if all(c_λ.>=0) && all(sep_vel_cond) && maintain_vel_cond
                if sum(m[1:n_contacts]) > max_cons
                    contactMode = m
                    max_cons = sum(m[1:n_contacts])
                end
            end
        end
    end
    
    return contactMode
end

compute_FA (generic function with 2 methods)

In [27]:
function compute_IV(x)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< 1e-6
    inactive_cs = abs.(a) .> 1e-6
    
    if sum(active_cs) == 0
        return zeros(Int, 4)
    end
    
    possibleModes = zeros(Bool, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end
    
    max_cons = 0
    for kk = 1:size(possibleModes, 1)
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, _ = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        dq_p, p_hat = computeResetMap(x, m)
        
        c_p_hat = contact_force_constraints(p_hat, m)
        As_eq, As_geq, _, _ = sliding_velocity_constraints(x, m)
        
        if all(c_p_hat.>=0) && all(A_separate*dq_p.>=0) && all(abs.(As_eq*dq_p).<1e-6) && all(As_geq*dq_p.>0)
             if sum(m[1:n_contacts]) > max_cons
                contactMode = m
                max_cons = sum(m[1:n_contacts])
            end
        end
    end
    
    return contactMode
end

compute_IV (generic function with 1 method)

In [65]:
function guard_conditions(x, contactMode, controller)
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    a[contactMode[1:n_contacts] .== 1] .= 0.0
    
    v_all = zeros(n_contacts)
    _, As_geq, _, _ = sliding_velocity_constraints(x, contactMode)
    v_all[1:size(As_geq,1)] = -As_geq*dq
    
    ddq, λ = solveEOM(x, contactMode, controller)
    c_λ = contact_force_constraints(λ, contactMode)
    c_λ_all = zeros(3*n_contacts)
    c_λ_all[1:length(c_λ)] = c_λ
    
    c = [a; v_all; c_λ_all]
    
    dir = [-ones(Int,length(a)); ones(Int,length(v_all)); ones(Int,length(c_λ_all))]
    
    return c, dir
end

guard_conditions (generic function with 2 methods)

In [66]:
function dynamics!(dx, x, p, t)
    # p from integrator: (vector n, contact mode, controller)
    q = x[1:3]
    dq = x[4:6]
    ddq, λ = solveEOM(x, p[1], p[2])
    dx .= [dq; ddq]
end

dynamics! (generic function with 1 method)

In [75]:
function conditions(out, x, t, integrator)
    contactMode = integrator.p[1]
    controller = integrator.p[2]
    c, dir = guard_conditions(x, contactMode, controller)
    out .= c
end

function affect!(integrator, idx)
    contactMode = integrator.p[1]
    x = integrator.u
    c, dir = guard_conditions(x, contactMode)
    
    # only consider upcrossing forces and constraints values(FA comp)
    # forces
    if dir[idx] > 0
        new_contactMode = compute_FA(x, integrator.p[2])
        integrator.p[1] .= reshape(new_contactMode,size(integrator.p[1]))
        println("New mode from FA: ", new_contactMode)
    end
    # constraints
#     if dir[idx] < 0
#         new_contactMode = contactMode
#         new_contactMode[idx] = false
#     end
end

function affect_neg!(integrator, idx)
    contactMode = integrator.p[1]
    x = integrator.u
    c, dir = guard_conditions(x, contactMode, integrator.p[2])
    
    # only consider down crossing constraint value(IV comp)
    if dir[idx] < 0
        new_contactMode = compute_IV(x)
        dq_p, p_hat = computeResetMap(x, new_contactMode)
        integrator.u .= [x[1:3]; dq_p]
        integrator.p[1] .= reshape(new_contactMode,size(integrator.p[1]))
        println("New mode from IV: ", new_contactMode)

    end
end

affect_neg! (generic function with 1 method)

In [76]:
function mode(x, controller)
    return compute_FA(x, controller)
end

mode (generic function with 1 method)

In [84]:
function no_controller(x, contactMode)
    return zeros(3)
end

function force_controller(x, contactMode)
    f = zeros(3)
    f[1] = 1.2*μ*m*g
    return f
end

function pd_controller(x, contactMode)
    # for sliding velocity
    q = x[1:3]
    dq = x[4:6]
    k = 10
    d = 1
    eq = q - q_ref
    eq[1] = 0
    f = -k.*eq .- d.*(dq - dq_ref)
    
    # add sliding force compensation
    if abs(f[1]) > 1e-3
        f[1] += μ*m*g
    end
    return f
end

function pd_pusher_controller(x, contactMode)
    # for sliding velocity
    pusher_p = [-w/2,0]
    q = x[1:3]
    dq = x[4:6]
    
    k = 10
    d = 1
    
    A = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    eq = q - q_ref
    eq[1] = 0
    fd = -k.*eq .- d.*(dq - dq_ref)
    
    # add sliding force compensation
    if abs(fd[1]) > 1e-3
        fd[1] += μ*m*g
    end
    
    c = A\fd
    f = A*c
    return f
end

function lqr_pusher_controller(x, contactMode)
    # TODO
    f = []
    return f
end
    
function hybrid_pusher_controller(x, contactMode)
    # TODO
    f = []
    return f
end
    

hybrid_pusher_controller (generic function with 1 method)

In [86]:
println(pd_pusher_controller(x0, initial_mode))
println(pd_controller(x0, initial_mode))

[5.9, -1.599333666587312, 0.799666833293656]
[5.9, -2.4991670832341404, -1.0]


In [87]:
tspan = (0.0, 5.0)
callback_length = 5*n_contacts

θ0 = 0.1
x0 = [0;h/2 + sin(θ0)*w/2 + 0.2;θ0;0;0;0]
q_ref = [0;h/2;0]
dq_ref = [1;0;0]
controller = pd_pusher_controller

initial_mode = mode(x0, controller)
prob = ODEProblem(dynamics!, x0, tspan, (initial_mode, controller))
cb = VectorContinuousCallback(conditions, affect!, affect_neg!, callback_length)
sol = solve(prob, Tsit5(); callback = cb, abstol=1e-15,reltol=1e-15)

New mode from IV: [1, 0, 0, 0]
New mode from IV: [1, 1, 1, 1]


retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 989-element Vector{Float64}:
 0.0
 0.00022231174525820041
 0.0005404073602531813
 0.0008792620284886741
 0.0013005752344500193
 0.0017617476016769534
 0.0022766746409535452
 0.0028313272445405165
 0.003425172821221457
 0.004049784289498922
 0.004700104078540059
 0.00537306132060961
 0.006064657916575022
 ⋮
 4.861784962368215
 4.874411599855697
 4.887082102581986
 4.899792890933445
 4.91252459573108
 4.925283992219284
 4.938102726492967
 4.950950832349927
 4.963828574551191
 4.97676595097265
 4.989703407420526
 5.0
u: 989-element Vector{Vector{Float64}}:
 [0.0, 0.6499167083234141, 0.1, 0.0, 0.0, 0.0]
 [1.457856071508608e-7, 0.6499164266525557, 0.1000001445142045, 0.0013114935114162307, -0.002533922410002669, 0.0012997584383133134]
 [8.613631699770176e-7, 0.6499150440938076, 0.10000085329162021, 0.0031875420623237926, -0.006158608101632765, 0.003155912089578983]
 [2.2799817757287527e-6, 0.6499123032017523, 0.10

In [40]:
function boxshape(q)
    p1 = q[1:2] + R_2D(q[3])*[w/2;h/2]
    p2 = q[1:2] + R_2D(q[3])*[w/2;-h/2]
    p3 = q[1:2] + R_2D(q[3])*[-w/2;-h/2]
    p4 = q[1:2] + R_2D(q[3])*[-w/2;h/2]
    pp = [p1 p2 p3 p4]
    return Shape(pp[1,:], pp[2,:])
end

boxshape (generic function with 1 method)

In [88]:
# animation 
n = length(sol.t)
x = zeros(n)
y = zeros(n)
θ = zeros(n)
for k = 1:n
    x[k] = sol.u[k][1]
    y[k] = sol.u[k][2]
    θ[k] = sol.u[k][3]
end
    
anim = @animate for i ∈ 1:n
    plot([-10,5],[0,0], lw = 2, c=:black, xlims=(-3,3), ylims=(-0.5,3))
    plot!(boxshape([x[i],y[i],θ[i]]), aspect_ratio=:equal, c=:gray, opacity=.5, legend=false)
    
end

Animation("/tmp/jl_NFHqHV", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000980.png", "000981.png", "000982.png", "000983.png", "000984.png", "000985.png", "000986.png", "000987.png", "000988.png", "000989.png"])

In [89]:
gif(anim, "anim.gif", fps = Int(floor(n/tspan[2])));

┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/anim.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114
